In [1]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
from search_videos_in_directory import search_videos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

[INFO] loading model...


In [ ]:
input_direc = input("Enter the absolute path of the input directory: ")
output_direc = input("Enter the absolute path of the output directory: ")
try:
    os.mkdir(output_direc)
except FileExistsError:
    pass
search_videos(input_direc, output_direc)

In [ ]:
for root, dirs, files in os.walk(output_direc):
    for file in files:

        try:
            im = Image.open(root + '/' + str(file))
            im.verify()
        except:
            print(str(file))
            os.remove(root + '/' + str(file))

In [2]:
input_direc = '/home/debanik/downloaded_videos'
output_direc = '/home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images_Deepfakes_only_100'

In [3]:
# create generator
datagen = ImageDataGenerator(samplewise_center=True, 
                             samplewise_std_normalization=True, 
                             rotation_range=10, 
                             horizontal_flip = True, 
                             validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64)
# # confirm the iterator works
# batchX, batchy = train_it.next()
# print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 199168 images belonging to 2 classes.


In [4]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [9]:
x = Input(shape = (256, 256, 3))

#ImageNet weights
model_mn = MobileNetV2(input_shape = (256,256,3),alpha=1.3,include_top=False)

#Use the generated model 
output_mn = model_mn(x)

#Add the fully-connected layers 
y = Flatten()(output_mn)
y = Dense(1000, activation = 'relu')(y)
y = Dense(1000, activation = 'relu')(y)
y = Dropout(0.5)(y)
y = Dense(1, activation='sigmoid')(y)


15441920/15441408 [==============================] - 6s 0us/step


In [10]:
model = Model(inputs = x, outputs = y)

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
mobilenetv2_1.30_224 (Model) (None, 8, 8, 1664)        3766048   
_________________________________________________________________
flatten_1 (Flatten)          (None, 106496)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              106497000 
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1001

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit_generator(train_it, epochs = 10)

W0623 17:58:53.133859 140448958920512 deprecation.py:323] From /home/debanik/PycharmProjects/Detecting-Deepfakes/venv_cpu/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
3112/3112 [==============================] - 35499s 11s/step - loss: 0.2082 - acc: 0.9282
Epoch 2/10
3112/3112 [==============================] - 35491s 11s/step - loss: 0.0594 - acc: 0.9773
Epoch 3/10
 329/3112 [==>...........................] - ETA: 8:49:58 - loss: 0.0483 - acc: 0.9815